In [5]:
import sys, os, io
import dvc.api
sys.path.append(os.path.abspath(os.path.join('..')))

In [6]:
from scripts.dataloader import DataLoader
from scripts.exploration import Exploration

In [7]:
train_data = '../data/train.csv'
test_data = '../data/test.csv'
store_data = '../data/store.csv'

### Load datasets from Gdrive

In [9]:
dataloader = DataLoader()
exp = Exploration()
repo = 'https://github.com/Haylemicheal/Pharmaceutical-Sales-Prediction/'
content_train =dvc.api.read(path=train_data, repo=repo, rev='v1')
df_train = dataloader.read_csv(io.StringIO(content_train))
content_store =dvc.api.read(path=store_data, repo=repo, rev='v1')
df_store = dataloader.read_csv(io.StringIO(content_store))
content_test =dvc.api.read(path=test_data, repo=repo, rev='v1')
df_test = dataloader.read_csv(io.StringIO(content_test))

In [10]:
df_train.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [11]:
df_train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday'],
      dtype='object')

In [12]:
df_train.dtypes

Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object

### Check for missing values

In [13]:
#Training data
percent_missing, missing_count = exp.get_missing(df_train)

In [14]:
percent_missing

0.0

In [15]:
missing_count 

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Fortunatly, We don't have a missing value in the training csv file

In [17]:
#Test data
percent_missing, missing_count = exp.get_missing(df_test)
missing_count

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

The test data has 11 missing values in the Open column

In [18]:
#Store data
percent_missing, missing_count = exp.get_missing(df_store)
missing_count

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

The store data has missing values on the following columns
- CompetitionDistance            3
- CompetitionOpenSinceMonth    354
- CompetitionOpenSinceYear     354
- Promo2SinceWeek              544
- Promo2SinceYear              544
- PromoInterval                544

### Handle the missing value in Store Dataset

In [19]:
df_store.dtypes

Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

In [30]:
df_clean_store = df_store.fillna(df_store.median(numeric_only=True))

In [31]:
df_clean_store['PromoInterval'] = df_clean_store['PromoInterval'].fillna(df_clean_store['PromoInterval'].mode()[0])

In [32]:
percent_missing, missing_count = exp.get_missing(df_clean_store)
missing_count

Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64

### Handle the missing value in Test Dataset

In [33]:
df_test.dtypes

Id                 int64
Store              int64
DayOfWeek          int64
Date              object
Open             float64
Promo              int64
StateHoliday      object
SchoolHoliday      int64
dtype: object

In [34]:
df_clean_test = df_test.fillna(df_test.mean(numeric_only=True))

In [35]:
percent_missing, missing_count = exp.get_missing(df_clean_test)
missing_count

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [39]:
df_clean_test.to_csv('../data/clean/test.csv')
df_train.to_csv('../data/clean/train.csv')
df_clean_store.to_csv('../data/clean/store.csv')

The above saved data will be tracked with dvc. The tag name for the above data is cleaned-v